In [ ]:
import pandas as pd
import numpy as np

In [ ]:
safedf1 = pd.read_csv("./data/safety.csv") # contain all the pairwise comparisons for safety 
safedf2 = pd.read_csv("./data/safetb.csv") # contain the pi and ni and ei for each id observation
wealthdf1 = pd.read_csv("./data/wealth.csv")
wealthf2 = pd.read_csv("./data/wealthtb.csv")
livelydf1 = pd.read_csv("./data/lively.csv")
livelydf2 = pd.read_csv("./data/livelytb.csv")

In [ ]:
#generate the two recorder 
#record the win or loss of each observaton
#recorder_po[i] record every id lost to i
#recorder_ne[i] record every id win i
def opponent_recorder(df1, df2, dict1, dict2): 
    num = len(df1.indices)
    recorder_po = [[] for i in range(num)]
    recorder_ne =  [[] for i in range(num)]

    for row in df2.iterrows():
        left_code = row[1]['left']
        right_code = row[1]['right']
        choi = row[1]['choice']
        m = dict1.get(left_code)
        n = dict1.get(right_code)
        if choi == "left":
            recorder_po[m].append(n)
            recorder_ne[n].append(m)
        #print(recorder_po[m])
        elif choi == "right":
            recorder_ne[m].append(n)
            recorder_po[n].append(m)
    return recorder_ne, recorder_po 

#generate the sum of rivals' pi value for every image id when this image win 
#generate the sum of rivals' ni value for every image id when this image lose
def oppoent_sum(df1, recorder_ne, recorder_po):
    num = len(df1.indices)
    posi_sum = []
    nega_sum = []
    for i in range(num):
        posum = 0
        nesum = 0
        for k1 in recorder_po[i]:
            val = dict2.get(k1)
            posum += val
        for k2 in recorder_ne[i]:
            val = 1 - dict2.get(k2)
            nesum += val
        posi_sum.append(posum)
        nega_sum.append(nesum)
    return posi_sum, nega_sum

#calculate the final qscore according to the formula
def qscore_cal(df):
    qscore_list = []
    for row in df.iterrows():
        score = 0
        p_num = row[1]['posi']
        n_num = row[1]['nega']
        pi = row[1]['pi']
        ni = row[1]['ni']
        po_sum = row[1]['posisum']
        ne_sum = row[1]['negasum']
        eq_num = row[1]['eq']
        ne_eq = ne_sum + eq_num
        score = pi + 1
        if p_num > 0:
            score += po_sum/p_num
        if ne_eq > 0:
            score -= ne_sum/ne_eq
        qscore = score * 10/3
        qscore_list.append(qscore)
    return qscore_list 

In [1]:
safedf1['indices'] = safedf1.index

dict1 = dict(zip(safedf1.id, safedf1.indices))
dict2 = dict(zip(safedf1.indices, safedf1.pi))

safe_ne_record, safe_po_record = opponent_recorder(safedf1, safedf2, dict1, dict2)
safe_posi_sum, safe_nega_sum = oppoent_sum(safedf1, safe_ne_record, safe_po_record)
safe_po_ne = pd.DataFrame(zip(safe_posi_sum, safe_nega_sum), columns = ["posisum", "negasum"])
safe_total = pd.concat([safedf1, safe_po_ne], axis = 1)
safe_qscore = qscore_cal(safe_total)
safe_total = pd.concat([safe_total, pd.DataFrame(safe_qscore, columns= ['qscore'])], axis = 1)
safe_total['total'] = safe_total['posi'] + safe_total['nega'] + safe_total['eq']
safe_total = safe_total[safe_total.total > 5] #filter to avoid unrepresentativeness
safe_loc = pd.read_csv("./safe_loc.csv") # this csv record lat and lon for every image id
safe_total = pd.merge(safe_total, safe_loc, left_on= 'id', right_on='id')
safe_total = safe_total.drop_duplicates()

sd = np.std(safe_total.qscore)
mean_score = np.mean(safe_total.qscore)
#calculate the thresholds for head and tail for class1 and class0
upper = mean_score + 1.8 * sd 
lower = mean_score - 1.8 * sd

high_safe = safe_total[safe_total.qscore > upper].sample(2000) #sample due for balanced classes
low_safe = safe_total[safe_total.qscore < lower]

#save the csv of two classes
high_safe.to_csv("./safe_high.csv")
low_safe.to_csv("./safe_low.csv")

NameError: name 'safedf1' is not defined

In [ ]:
wealthdf1['indices'] = wealthdf1.index

dict1 = dict(zip(wealthdf1.id, wealthdf1.indices))
dict2 = dict(zip(wealthdf1.indices, wealthdf1.pi))

wealth_ne_record, wealth_po_record = opponent_recorder(wealthdf1, wealthdf2, dict1, dict2)
wealth_posi_sum, wealth_nega_sum = oppoent_sum(wealthdf1, wealth_ne_record, wealth_po_record)
wealth_po_ne = pd.DataFrame(zip(wealth_posi_sum, wealth_nega_sum), columns = ["posisum", "negasum"])
wealth_total = pd.concat([wealthdf1, wealth_po_ne], axis = 1)
wealth_qscore = qscore_cal(wealth_total)
wealth_total = pd.concat([wealth_total, pd.DataFrame(wealth_qscore, columns= ['qscore'])], axis = 1)
wealth_total['total'] = wealth_total['posi'] + wealth_total['nega'] + wealth_total['eq']
wealth_total = wealth_total[wealth_total.total > 5]
wealth_loc = pd.read_csv("./wealth_loc.csv")
wealth_total = pd.merge(wealth_total, wealth_loc, left_on= 'id', right_on='id')
wealth_total = wealth_total.drop_duplicates()

sd = np.std(wealth_total.qscore)
mean_score = np.mean(wealth_total.qscore)
upper = mean_score + 1.8 * sd
lower = mean_score - 1.8 * sd

high_wealth = wealth_total[wealth_total.qscore > upper].sample(2000)
low_wealth = wealth_total[wealth_total.qscore < lower]
high_wealth.to_csv("./wealth_high.csv")
low_wealth.to_csv("./wealth_low.csv")

In [ ]:
livelydf1['indices'] = livelydf1.index

dict1 = dict(zip(livelydf1.id, livelydf1.indices))
dict2 = dict(zip(livelydf1.indices, livelydf1.pi))

lively_ne_record, lively_po_record = opponent_recorder(livelydf1, livelydf2, dict1, dict2)
lively_posi_sum, lively_nega_sum = oppoent_sum(livelydf1, lively_ne_record, lively_po_record)
lively_po_ne = pd.DataFrame(zip(lively_posi_sum, lively_nega_sum), columns = ["posisum", "negasum"])
lively_total = pd.concat([livelydf1, lively_po_ne], axis = 1)
lively_qscore = qscore_cal(lively_total)
lively_total = pd.concat([lively_total, pd.DataFrame(lively_qscore, columns= ['qscore'])], axis = 1)
lively_total['total'] = lively_total['posi'] + lively_total['nega'] + lively_total['eq']
lively_total = lively_total[lively_total.total > 5]
lively_loc = pd.read_csv("./lively_loc.csv")
lively_total = pd.merge(lively_total, lively_loc, left_on= 'id', right_on='id')
lively_total = lively_total.drop_duplicates()

sd = np.std(lively_total.qscore)
mean_score = np.mean(lively_total.qscore)
upper = mean_score + 1.8 * sd
lower = mean_score - 1.8 * sd

high_lively = lively_total[lively_total.qscore > upper].sample(2000)
low_lively = lively_total[lively_total.qscore < lower]
high_lively.to_csv("./lively_high.csv")
low_lively.to_csv("./lively_low.csv")